# Iterators

## Iterator & Iterable

* An iterator is any value that implements `std::iter::Iterator` trait:

In [6]:
mod explain {
    trait Iterator {
        type Item;
        fn next(&mut self) -> Option<Self::Item>;
        //... many default methods
    }
}

* If there’s a natural way to iterate over some type, that type can implement `std::iter::IntoIterator`, whose `into_iter` method takes a value and returns an iterator over it:

In [7]:
mod explain {
    trait IntoIterator {
        type Item;
        type IntoIter: Iterator;
        fn into_iter(self) -> Self::IntoIter;
    }
}

* An **iterator** is any type that implements `Iterator`.

* An **iterable** is any type that implements `IntoIterator`: you can get an iterator over it by calling its `into_iter` method. The vector reference `&data` is the iterable in this case.

* An iterator **produces** values.

* The values an iterator produces are **items**.

* The code that receives the items an iterator produces is the **consumer** (the for loop is the consumer).

## for loop

In [8]:
let data = vec!["Fender", "Gibson", "PRS", "Yamaha"];

for element in &data {
    println!("{}", element);
}

Fender
Gibson
PRS
Yamaha


()

* for loop is a shorthand for:

In [9]:
{
    let mut iterator = (&data).into_iter();
    while let Some(element) = iterator.next() {
        println!("{}", element);
    }
}

Fender
Gibson
PRS
Yamaha


()

## Creating iterators

### iter() and iter_mut()

* Most collection types and array slices like `&[T]` and `&mut [T]` provide `iter()` and `iter_mut()` methods that return the natural iterators over the type, producing a shared or mutable reference to each item

In [10]:
{ 
    let data = vec![1, 1, 2, 3, 5, 8, 13];

    let mut iterator = data.iter();
    assert_eq!(iterator.next(), Some(&1));
    assert_eq!(iterator.next(), Some(&1));
    assert_eq!(iterator.next(), Some(&2));
    assert_eq!(iterator.next(), Some(&3));
    assert_eq!(iterator.next(), Some(&5));
    assert_eq!(iterator.next(), Some(&8));
    assert_eq!(iterator.next(), Some(&13));
    assert_eq!(iterator.next(), None);
}

()

* This iterator’s item type is `&i32`: each call to next produces a reference to the next element, until we reach the end of the vector.

### Iteration over &str

* There is no `iter()` method on the `&str` string slice type. Instead, if `s` is a `&str`, then `s.bytes()` returns an iterator that produces each byte of s, whereas `s.chars()` interprets the contents as UTF-8 and produces each Unicode character.

### IntoIterator implementations

* When a type implements `IntoIterator`, you can call its `into_iter()` method yourself, just as a for loop would:

In [11]:
use std::collections::BTreeSet;

let mut guitars = BTreeSet::new();
guitars.insert("Yamaha".to_string());
guitars.insert("Fender".to_string());
guitars.insert("Gibson".to_string());

let mut iterator = guitars.into_iter();
assert_eq!(iterator.next(), Some("Fender".to_string()));
assert_eq!(iterator.next(), Some("Gibson".to_string()));
assert_eq!(iterator.next(), Some("Yamaha".to_string()));
assert_eq!(iterator.next(), None);

* Most collections provide several implementations of `IntoIterator`, for shared references `(&T)`, mutable references `(&mut T)`, and moves `(T)`:
  * Given a shared reference to the collection, `into_iter()` returns an iterator that produces shared references to its items. For example, in the preceding code, `(&favorites).into_iter()` would return an iterator whose `Item` type is `&String`.
  * Given a mutable reference to the collection, `into_iter()` returns an iterator that produces mutable references to the items. For example, if vector is some `Vec<String>`, the call `(&mut vector).into_iter()` returns an iterator whose `Item` type is `&mut String`.
  * When passed the collection by value, `into_iter()` returns an iterator that takes ownership of the collection and returns items by value; the items’ ownership moves from the collection to the consumer, and the original collection is consumed in the process. For example, the call `favorites.into_iter()` in the preceding code returns an iterator that produces each string by value; the consumer receives ownership of each string. When the iterator is dropped, any elements remaining in the `BTreeSet` are dropped too, and the set’s now-empty husk is disposed of.

* Since a for loop applies `IntoIterator::into_iter()` to its operand, these three implementations are what create the following idioms for iterating over shared or mutable references to a collection, or consuming the collection and taking ownership of its elements:

In [12]:
use std::collections::BTreeSet;

let mut guitars = vec!["Yamaha".to_string(), "Fender".to_string(), "Gibson".to_string()];

for guitar in &guitars {
    println!("{}", guitar);
}

for guitar in &mut guitars {
    *guitar += "!";
}

for guitar in guitars {
    println!("{}", guitar);
}



Yamaha
Fender
Gibson
Yamaha!
Fender!
Gibson!


()

In [13]:
fn dump<T, U>(t: T)
    where T: IntoIterator<Item = U>,
          U: std::fmt::Debug {
    for u in t {
        println!("{:?}", u);
    }
}

In [15]:
let mut guitars = vec!["Yamaha".to_string(), "Fender".to_string(), "Gibson".to_string()];
dump(guitars);

"Yamaha"
"Fender"
"Gibson"


### from_fn

In [16]:
:dep rand = "0.7"

In [19]:
use rand::random;
use std::iter::from_fn;

let lengths: Vec<f64> = from_fn(|| Some((random::<f64>() - random::<f64>()).abs()))
    .take(10)
    .collect();

In [20]:
lengths

[0.6025133528294632, 0.6864637246723242, 0.6252755365365168, 0.3727390601424685, 0.24371532293787768, 0.1895236212725533, 0.1477553569015797, 0.11839366878974455, 0.34984111734947776, 0.18264228921097836]

In [21]:
fn fibonacci() -> impl Iterator<Item = usize> {
    let mut state = (0, 1);
    
    from_fn(move || {
        let next = state.0;
        state = (state.1, state.0 + state.1);
        Some(next)
    })
}

In [22]:
fibonacci().take(10).collect::<Vec<_>>()

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

### drain methods

* The `drain()` method that takes a mutable reference to the collection and returns an iterator that passes ownership of each element to the consumer
  * However, unlike the `into_iter()` method, which takes the collection by value and consumes it, drain merely borrows a mutable reference to the collection, and when the iterator is dropped, it removes any remaining elements from the collection and leaves it empty

In [26]:
let mut data = vec![1, 2, 3, 4, 5, 6, 7, 8, 9, 10];
let inner = data.drain(3..7).collect::<Vec<i32>>();

println!("{:?}", data);
println!("{:?}", inner);

[1, 2, 3, 8, 9, 10]
[4, 5, 6, 7]


### Other iterator sources

#### std::ops::Range

In [31]:
(1..10).collect::<Vec<i32>>()

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [34]:
(1..10).step_by(2).collect::<Vec<i32>>()

[1, 3, 5, 7, 9]

#### Vec[T], &[T]

* `v.windows(3)`

In [36]:
let v = vec![1, 2, 3, 4, 5, 6, 7, 8, 9, 10];

for wnd in v.windows(3) {
    println!("{:?}", wnd);
}

[1, 2, 3]
[2, 3, 4]
[3, 4, 5]
[4, 5, 6]
[5, 6, 7]
[6, 7, 8]
[7, 8, 9]
[8, 9, 10]


()

* `v.chunks(3)` & `v.chunks_mut(3)`

In [37]:
for chunk in v.chunks(3) {
    println!("{:?}", chunk);
}

[1, 2, 3]
[4, 5, 6]
[7, 8, 9]
[10]


()

* `v.split(predicate)` & `v.split_mut(predicate)`

In [41]:
for slice in v.split(|&n| n % 3 == 0) {
    println!("{:?}", slice);
}

[1, 2]
[4, 5]
[7, 8]
[10]


()

In [42]:
for slice in v.rsplit(|&n| n % 3 == 0) {
    println!("{:?}", slice);
}

[10]
[7, 8]
[4, 5]
[1, 2]


()

#### String, &str

In [47]:
let text = "Nobody expects the Spanish Inquisition!\n Amongst our weaponry are such diverse elements as fear, surprise, ruthless efficiency, an almost fanatical devotion to the Pope, and nice red uniforms.".to_string();

* `text.bytes()` - produces bytes of the UTF-8 form

In [48]:
text.bytes()

Bytes(Copied { it: Iter([78, 111, 98, 111, 100, 121, 32, 101, 120, 112, 101, 99, 116, 115, 32, 116, 104, 101, 32, 83, 112, 97, 110, 105, 115, 104, 32, 73, 110, 113, 117, 105, 115, 105, 116, 105, 111, 110, 33, 10, 32, 65, 109, 111, 110, 103, 115, 116, 32, 111, 117, 114, 32, 119, 101, 97, 112, 111, 110, 114, 121, 32, 97, 114, 101, 32, 115, 117, 99, 104, 32, 100, 105, 118, 101, 114, 115, 101, 32, 101, 108, 101, 109, 101, 110, 116, 115, 32, 97, 115, 32, 102, 101, 97, 114, 44, 32, 115, 117, 114, 112, 114, 105, 115, 101, 44, 32, 114, 117, 116, 104, 108, 101, 115, 115, 32, 101, 102, 102, 105, 99, 105, 101, 110, 99, 121, 44, 32, 97, 110, 32, 97, 108, 109, 111, 115, 116, 32, 102, 97, 110, 97, 116, 105, 99, 97, 108, 32, 100, 101, 118, 111, 116, 105, 111, 110, 32, 116, 111, 32, 116, 104, 101, 32, 80, 111, 112, 101, 44, 32, 97, 110, 100, 32, 110, 105, 99, 101, 32, 114, 101, 100, 32, 117, 110, 105, 102, 111, 114, 109, 115, 46]) })

* `text.chars()` - produces `char`s of UTF-8 represents

In [49]:
text.chars()

Chars(['N', 'o', 'b', 'o', 'd', 'y', ' ', 'e', 'x', 'p', 'e', 'c', 't', 's', ' ', 't', 'h', 'e', ' ', 'S', 'p', 'a', 'n', 'i', 's', 'h', ' ', 'I', 'n', 'q', 'u', 'i', 's', 'i', 't', 'i', 'o', 'n', '!', '\n', ' ', 'A', 'm', 'o', 'n', 'g', 's', 't', ' ', 'o', 'u', 'r', ' ', 'w', 'e', 'a', 'p', 'o', 'n', 'r', 'y', ' ', 'a', 'r', 'e', ' ', 's', 'u', 'c', 'h', ' ', 'd', 'i', 'v', 'e', 'r', 's', 'e', ' ', 'e', 'l', 'e', 'm', 'e', 'n', 't', 's', ' ', 'a', 's', ' ', 'f', 'e', 'a', 'r', ',', ' ', 's', 'u', 'r', 'p', 'r', 'i', 's', 'e', ',', ' ', 'r', 'u', 't', 'h', 'l', 'e', 's', 's', ' ', 'e', 'f', 'f', 'i', 'c', 'i', 'e', 'n', 'c', 'y', ',', ' ', 'a', 'n', ' ', 'a', 'l', 'm', 'o', 's', 't', ' ', 'f', 'a', 'n', 'a', 't', 'i', 'c', 'a', 'l', ' ', 'd', 'e', 'v', 'o', 't', 'i', 'o', 'n', ' ', 't', 'o', ' ', 't', 'h', 'e', ' ', 'P', 'o', 'p', 'e', ',', ' ', 'a', 'n', 'd', ' ', 'n', 'i', 'c', 'e', ' ', 'r', 'e', 'd', ' ', 'u', 'n', 'i', 'f', 'o', 'r', 'm', 's', '.'])

* `text.split_whitespace()`

In [51]:
for text_chunk in text.split_whitespace() {
    println!("{}", text_chunk);
}

Nobody
expects
the
Spanish
Inquisition!
Amongst
our
weaponry
are
such
diverse
elements
as
fear,
surprise,
ruthless
efficiency,
an
almost
fanatical
devotion
to
the
Pope,
and
nice
red
uniforms.


()

* `text.lines()`

In [54]:
for line in text.lines() {
    println!("{}", line.trim());
}

Nobody expects the Spanish Inquisition!
Amongst our weaponry are such diverse elements as fear, surprise, ruthless efficiency, an almost fanatical devotion to the Pope, and nice red uniforms.


()

* `text.matches`

In [56]:
let text = "A1B2C3D4".to_string();

for numeric_chunks in text.matches(char::is_numeric) {
    println!("{}", numeric_chunks);
}

1
2
3
4


()

#### HashMap, BTreeMap

In [57]:
let mut dictionary = std::collections::HashMap::new();

dictionary.insert("apple", 3);
dictionary.insert("banana", 2);
dictionary.insert("cherry", 5);

In [58]:
dictionary.keys()

["cherry", "banana", "apple"]

In [59]:
dictionary.values()

[5, 2, 3]

#### HashSet, BTreeSet

In [ ]:
let dictionary_a = std::collections::HashMap::new();



#### Free functions

In [68]:
let empty_iterable = std::iter::empty::<i32>();

let mut iter = empty_iterable.into_iter();

assert_eq!(iter.next(), None);

In [69]:
let one_element_iterable = std::iter::once(42);

let mut iter = one_element_iterable.into_iter();

assert_eq!(iter.next(), Some(42));
assert_eq!(iter.next(), None);

In [3]:
let pattern = std::iter::repeat("*-@").take(9).collect::<String>();
pattern

"*-@*-@*-@*-@*-@*-@*-@*-@*-@"

## Iterator Adaptors

* the `Iterator` trait provides a broad selection of adapter methods, or simply adapters, that consume one iterator and build a new one with useful behaviors

### map & filter

In [16]:
let text = "Nobody expects the Spanish Inquisition!\n     Amongst our weaponry are such diverse elements as \n   fear, \n   surprise, \n   ruthless efficiency, \n   an almost fanatical devotion to the Pope, and nice red uniforms.".to_string();

let text_lines = text.lines()
    .map(|s| s.trim().to_owned())
    .collect::<Vec<String>>();

text_lines

["Nobody expects the Spanish Inquisition!", "Amongst our weaponry are such diverse elements as", "fear,", "surprise,", "ruthless efficiency,", "an almost fanatical devotion to the Pope, and nice red uniforms."]

In [17]:
let short_lines = text.lines()
    .map(|s| s.trim().to_owned())
    .filter(|s| s.len() < 20)
    .collect::<Vec<String>>();

short_lines

["fear,", "surprise,"]

* Adapters are lazy

In [24]:
["one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten"]
    .iter()
    .map(|s| println!("{}", s));

### filter_map

In [27]:
let input_data = "1\n2\n3\n4\n5\nabc\n6\n7\ndefn8\n9\n10";

let numbers = input_data.lines()
    .filter_map(|line| line.parse::<i32>().ok())
    .collect::<Vec<i32>>();

numbers

[1, 2, 3, 4, 5, 6, 7, 9, 10]

### flat_map

In [38]:
{
   use std::collections::HashMap;

    let mut major_cities = HashMap::new();
    major_cities.insert("Japan", vec!["Tokyo", "Kyoto"]);
    major_cities.insert("The United States", vec!["Portland", "Nashville"]);
    major_cities.insert("Brazil", vec!["São Paulo", "Brasília"]);
    major_cities.insert("Kenya", vec!["Nairobi", "Mombasa"]);
    major_cities.insert("The Netherlands", vec!["Amsterdam", "Utrecht"]);
    
    let countries = ["Japan", "Brazil", "Kenya"];
    
    let selected_cities = countries.iter()
        .flat_map(|country| &major_cities[country])
        .collect::<Vec<&&str>>();

    println!("{:?}", selected_cities);

}

["Tokyo", "Kyoto", "São Paulo", "Brasília", "Nairobi", "Mombasa"]


()

### flatten

In [51]:
{
    let all_cities: Vec<&&str> = major_cities
    .values()
    .flatten()
    .collect();

    println!("{:?}", all_cities);
}

["Tokyo", "Kyoto", "Amsterdam", "Utrecht", "São Paulo", "Brasília", "Portland", "Nashville", "Nairobi", "Mombasa"]


()

### take & take_while

In [7]:
let message = "To: jimb\r\n\
               From: superego <editor@oreilly.com>\r\n\
               \r\n\
               Did you get any writing done today?\r\n\
               When will you stop wasting time plotting fractals?\r\n";

message.lines().take_while(|l| !l.is_empty()).collect::<Vec<&str>>()

["To: jimb", "From: superego <editor@oreilly.com>"]

### skip & skip_while

In [8]:
let message = "To: jimb\r\n\
               From: superego <editor@oreilly.com>\r\n\
               \r\n\
               Did you get any writing done today?\r\n\
               When will you stop wasting time plotting fractals?\r\n";

message
    .lines()
    .skip_while(|l| !l.is_empty())
    .skip(1)
    .collect::<Vec<&str>>()

["Did you get any writing done today?", "When will you stop wasting time plotting fractals?"]

### peekable

* A peekable iterator lets you peek at the next item that will be produced without actually consuming it.

In [10]:
use std::iter::Peekable;

fn parse_number<I>(tokens: &mut Peekable<I>) -> i32
    where I: Iterator<Item = char> 
{    
    let mut number = 0;

    loop {
        match tokens.peek() {
            Some(c) if c.is_digit(10) => {
                number = number * 10 + c.to_digit(10).unwrap() as i32;                
            }
            _ => return number
        }
        tokens.next();
    }
}

let mut chars = "423143,353454".chars().peekable();
assert_eq!(parse_number(&mut chars), 423143);
assert_eq!(chars.next(), Some(','));
assert_eq!(parse_number(&mut chars), 353454);
assert_eq!(chars.next(), None);

### fuse

* Once an `Iterator` has returned `None`, the trait doesn’t specify how it ought to behave if you call its next method again. Most iterators just return `None` again, but not all. If your code counts on that behavior, you may be in for a surprise.
* The fuse adapter takes any iterator and produces one that will definitely continue to return `None` once it has done so the first time:

In [11]:
struct Flaky(bool);

impl Iterator for Flaky {
    type Item = &'static str;
    fn next(&mut self) -> Option<Self::Item> {
        if self.0 {
            self.0 = false;
            Some("totally the last item")
        } else {
            self.0 = true; // D'oh!
            None
        }
    }
}

let mut flaky = Flaky(true);
assert_eq!(flaky.next(), Some("totally the last item"));
assert_eq!(flaky.next(), None);
assert_eq!(flaky.next(), Some("totally the last item"));

let mut not_flaky = Flaky(true).fuse();
assert_eq!(not_flaky.next(), Some("totally the last item"));
assert_eq!(not_flaky.next(), None);
assert_eq!(not_flaky.next(), None);

### Reversible Iterators & rev

* The `rev` adapter takes any iterator and produces one that produces the same items in reverse order
* The `DoubleEndedIterator` trait extends `Iterator` with a `next_back` method that returns the last item of the iterator, or `None` if the iterator is empty.

In [14]:
mod explain {
    trait DoubleEndedIterator: Iterator {
        fn next_back(&mut self) -> Option<Self::Item>;
        //... many default methods
    }
}

In [18]:
fn main() {
    let mut data = vec![1, 2, 3, 4];
    
    let mut iterator = data.iter().rev();
    assert_eq!(iterator.next(), Some(&4));
    assert_eq!(iterator.next(), Some(&3));
    assert_eq!(iterator.next(), Some(&2));
    assert_eq!(iterator.next(), Some(&1));
    assert_eq!(iterator.next(), None);
}

main();